In [62]:
#importing required libraries
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# to get the similarity scores for large data sets using linear kernel over cosine_sim
from sklearn.metrics.pairwise import linear_kernel

In [63]:
#importing data into python environment and creating data frame

movie_df=pd.read_csv('db_5000_movies.txt')
credit_df=pd.read_csv('db_5000_credits.txt')

In [64]:
movie_df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [65]:
credit_df.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [66]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [67]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [68]:
movie_df.rename(columns={'id':'movie_id'},inplace=True)

In [69]:
final_df=movie_df.merge(credit_df,on='movie_id')

In [70]:
final_df.head(2)

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [71]:
#dropping duplicate and unwanted columns
final_df=final_df.drop(columns=['title_y','title_x','homepage','budget','original_language','runtime'])


In [72]:
final_df.head(2)

,genres,movie_id,keywords,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,status,tagline,vote_average,vote_count,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [73]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                4803 non-null   object 
 1   movie_id              4803 non-null   int64  
 2   keywords              4803 non-null   object 
 3   original_title        4803 non-null   object 
 4   overview              4800 non-null   object 
 5   popularity            4803 non-null   float64
 6   production_companies  4803 non-null   object 
 7   production_countries  4803 non-null   object 
 8   release_date          4802 non-null   object 
 9   revenue               4803 non-null   int64  
 10  spoken_languages      4803 non-null   object 
 11  status                4803 non-null   object 
 12  tagline               3959 non-null   object 
 13  vote_average          4803 non-null   float64
 14  vote_count            4803 non-null   int64  
 15  cast                 

In [74]:
final_df=final_df.drop(columns=['status','release_date','production_countries','production_companies','spoken_languages'])

In [75]:
final_df.head(2)

,genres,movie_id,keywords,original_title,overview,popularity,revenue,tagline,vote_average,vote_count,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2787965087,Enter the World of Pandora.,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,961000000,"At the end of the world, the adventure begins.",6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [76]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   genres          4803 non-null   object 
 1   movie_id        4803 non-null   int64  
 2   keywords        4803 non-null   object 
 3   original_title  4803 non-null   object 
 4   overview        4800 non-null   object 
 5   popularity      4803 non-null   float64
 6   revenue         4803 non-null   int64  
 7   tagline         3959 non-null   object 
 8   vote_average    4803 non-null   float64
 9   vote_count      4803 non-null   int64  
 10  cast            4803 non-null   object 
 11  crew            4803 non-null   object 
dtypes: float64(2), int64(3), object(7)
memory usage: 487.8+ KB


In [77]:
#  have some missing values in overview column , so replacing with empty string.
final_df['overview']=final_df['overview'].fillna('')
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   genres          4803 non-null   object 
 1   movie_id        4803 non-null   int64  
 2   keywords        4803 non-null   object 
 3   original_title  4803 non-null   object 
 4   overview        4803 non-null   object 
 5   popularity      4803 non-null   float64
 6   revenue         4803 non-null   int64  
 7   tagline         3959 non-null   object 
 8   vote_average    4803 non-null   float64
 9   vote_count      4803 non-null   int64  
 10  cast            4803 non-null   object 
 11  crew            4803 non-null   object 
dtypes: float64(2), int64(3), object(7)
memory usage: 487.8+ KB


In [78]:
## Now our data is ready and we will build a recommendation system to recommend movie based on overview
tfidf=TfidfVectorizer(stop_words='english',ngram_range=(1,3),analyzer='word')
tfidf_matrix=tfidf.fit_transform(final_df['overview'])
tfidf_matrix.shape

(4803, 261395)

In [79]:
## Now we will take similarity scores for each movie with rest based on overview
sim_score=linear_kernel(tfidf_matrix,tfidf_matrix)
sim_score.shape

(4803, 4803)

In [80]:
sim_score[1]

array([0.        , 1.        , 0.        , ..., 0.00488271, 0.        ,
       0.        ])

In [81]:
# creating a series to get index based on movie title

movie_index=pd.Series(final_df.index,index=final_df['original_title']).drop_duplicates()
movie_index[:5]
    

original_title
Avatar                                      0
Pirates of the Caribbean: At World's End    1
Spectre                                     2
The Dark Knight Rises                       3
John Carter                                 4
dtype: int64

In [82]:
def get_sim_movies(title,sim_score):
    index=movie_index[title]
    score=list(enumerate(sim_score[index]))
    score=sorted(score,key=lambda x: x[1], reverse=True)
    score=score[1:11]
    indices=[i[0] for i in score]
    
    return final_df['original_title'].iloc[indices]

In [83]:
print(get_sim_movies('Avatar',sim_score))

634                       The Matrix
3604                       Apollo 18
1341                Obitaemyy Ostrov
529                 Tears of the Sun
2130                    The American
775                        Supernova
847                         Semi-Pro
311     The Adventures of Pluto Nash
1610                           Hanna
1213     Aliens vs Predator: Requiem
Name: original_title, dtype: object


In [84]:
print(get_sim_movies('Batman Forever',sim_score))

3                         The Dark Knight Rises
1359                                     Batman
65                              The Dark Knight
119                               Batman Begins
210                              Batman & Robin
428                              Batman Returns
3854    Batman: The Dark Knight Returns, Part 2
2507                                  Slow Burn
1281                                    Hackers
1181                                        JFK
Name: original_title, dtype: object


## Version 2 --> Enhancing the recommendation system including director, top3 actors , genres,keywords in meta data and finding similarity based on that


In [85]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   genres          4803 non-null   object 
 1   movie_id        4803 non-null   int64  
 2   keywords        4803 non-null   object 
 3   original_title  4803 non-null   object 
 4   overview        4803 non-null   object 
 5   popularity      4803 non-null   float64
 6   revenue         4803 non-null   int64  
 7   tagline         3959 non-null   object 
 8   vote_average    4803 non-null   float64
 9   vote_count      4803 non-null   int64  
 10  cast            4803 non-null   object 
 11  crew            4803 non-null   object 
dtypes: float64(2), int64(3), object(7)
memory usage: 487.8+ KB


In [86]:
## converting features into corresponding objects

features=['cast','crew','genres','keywords']

from ast import literal_eval

for feature in features:
    final_df[feature]=final_df[feature].apply(literal_eval)

In [87]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   genres          4803 non-null   object 
 1   movie_id        4803 non-null   int64  
 2   keywords        4803 non-null   object 
 3   original_title  4803 non-null   object 
 4   overview        4803 non-null   object 
 5   popularity      4803 non-null   float64
 6   revenue         4803 non-null   int64  
 7   tagline         3959 non-null   object 
 8   vote_average    4803 non-null   float64
 9   vote_count      4803 non-null   int64  
 10  cast            4803 non-null   object 
 11  crew            4803 non-null   object 
dtypes: float64(2), int64(3), object(7)
memory usage: 487.8+ KB


In [88]:
def get_director(x):
    for crew in x:
        if crew['job']=='Director':
            return crew['name']
    return np.nan
        

In [89]:
final_df['Director']=final_df['crew'].apply(get_director)
final_df.head(2)

,genres,movie_id,keywords,original_title,overview,popularity,revenue,tagline,vote_average,vote_count,cast,crew,Director
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2787965087,Enter the World of Pandora.,7.2,11800,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de...",James Cameron
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,961000000,"At the end of the world, the adventure begins.",6.9,4500,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de...",Gore Verbinski


In [90]:

# function to return top 3 crew 
def get_top3(x):
    if isinstance(x,list):
        names=[i['name'] for i in x]
        if len(names)>3:
            names=names[:3]
        return names
    return []

In [91]:
features=['genres','keywords','cast']
for feature in features:
    final_df[feature]=final_df[feature].apply(get_top3)

In [92]:
final_df.head(2)

,genres,movie_id,keywords,original_title,overview,popularity,revenue,tagline,vote_average,vote_count,cast,crew,Director
0,"[Action, Adventure, Fantasy]",19995,"[culture clash, future, space war]",Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2787965087,Enter the World of Pandora.,7.2,11800,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{'credit_id': '52fe48009251416c750aca23', 'de...",James Cameron
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island]",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,961000000,"At the end of the world, the adventure begins.",6.9,4500,"[Johnny Depp, Orlando Bloom, Keira Knightley]","[{'credit_id': '52fe4232c3a36847f800b579', 'de...",Gore Verbinski


In [93]:
def format_data(x):
    if isinstance(x,list):
        return [str.lower(i.replace(" ","")) for i in x]
    else:
        if isinstance(x,str):
            return str.lower(x.replace(" ",""))
        else:
            return ''
        
## function to convert to lower case and remove space between words
                             
    

In [94]:
features=['genres','keywords','cast','Director']

for feature in features:
    final_df[feature]=final_df[feature].apply(format_data)

In [95]:
final_df.head(3)

,genres,movie_id,keywords,original_title,overview,popularity,revenue,tagline,vote_average,vote_count,cast,crew,Director
0,"[action, adventure, fantasy]",19995,"[cultureclash, future, spacewar]",Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2787965087,Enter the World of Pandora.,7.2,11800,"[samworthington, zoesaldana, sigourneyweaver]","[{'credit_id': '52fe48009251416c750aca23', 'de...",jamescameron
1,"[adventure, fantasy, action]",285,"[ocean, drugabuse, exoticisland]",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,961000000,"At the end of the world, the adventure begins.",6.9,4500,"[johnnydepp, orlandobloom, keiraknightley]","[{'credit_id': '52fe4232c3a36847f800b579', 'de...",goreverbinski
2,"[action, adventure, crime]",206647,"[spy, basedonnovel, secretagent]",Spectre,A cryptic message from Bond’s past sends him o...,107.376788,880674609,A Plan No One Escapes,6.3,4466,"[danielcraig, christophwaltz, léaseydoux]","[{'credit_id': '54805967c3a36829b5002c41', 'de...",sammendes


In [97]:
def form_metadata(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['Director'] + ' ' + ' '.join(x['genres'])

In [101]:
final_df['metadata']=final_df.apply(form_metadata,axis=1)


In [102]:
final_df['metadata'].head()

0    cultureclash future spacewar samworthington zo...
1    ocean drugabuse exoticisland johnnydepp orland...
2    spy basedonnovel secretagent danielcraig chris...
3    dccomics crimefighter terrorist christianbale ...
4    basedonnovel mars medallion taylorkitsch lynnc...
Name: metadata, dtype: object

In [103]:
## using countvectorizer as tfidf will reduce the importance for directors 
#and actors if they come in multiple movies 

cv=CountVectorizer(stop_words='english')
matrix=cv.fit_transform(final_df['metadata'])

matrix.shape



(4803, 11520)

In [104]:
sim_matrix=linear_kernel(matrix,matrix)
sim_matrix.shape

(4803, 4803)

In [111]:
def get_recommendations(title,sim_matrix):
    idx=movie_index[title]
    sim_score=list(enumerate(sim_matrix[idx]))
    sim_score=sorted(sim_score,key=lambda x:x[1],reverse=True)
    sim_score= sim_score[1:11] ## taking top 10 similar movies
    indices=[i[0] for i in sim_score]
    return final_df['original_title'].iloc[indices]
    
    

In [112]:
print(get_recommendations('Avatar',sim_matrix))

206                            Clash of the Titans
1         Pirates of the Caribbean: At World's End
5                                     Spider-Man 3
9               Batman v Superman: Dawn of Justice
10                                Superman Returns
12      Pirates of the Caribbean: Dead Man's Chest
14                                    Man of Steel
17     Pirates of the Caribbean: On Stranger Tides
19       The Hobbit: The Battle of the Five Armies
20                          The Amazing Spider-Man
Name: original_title, dtype: object


In [113]:
print(get_sim_movies('Batman Forever',sim_matrix))

210                        Batman & Robin
3                   The Dark Knight Rises
9      Batman v Superman: Dawn of Justice
10                       Superman Returns
14                           Man of Steel
65                        The Dark Knight
72                          Suicide Squad
98      The Hobbit: An Unexpected Journey
119                         Batman Begins
303                              Catwoman
Name: original_title, dtype: object
